# House Prices - Advanced Regression Techniques


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost

In [2]:
df_train = pd.read_csv("data/train.csv")

In [3]:
df_test = pd.read_csv("data/test.csv")

In [4]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_train.head())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story            7          

In [5]:
df_train.shape

(1460, 81)

In [6]:
df_test.shape

(1459, 80)

In [7]:
print("Percentage of null values for df_train")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    percentage_null_values = df_train.isnull().sum()*100/df_train.shape[0]
    
    for col,val in percentage_null_values.iteritems():
        if val!=0:
            print(col,"    -    ",val)

Percentage of null values for df_train
LotFrontage     -     17.73972602739726
Alley     -     93.76712328767124
MasVnrType     -     0.547945205479452
MasVnrArea     -     0.547945205479452
BsmtQual     -     2.5342465753424657
BsmtCond     -     2.5342465753424657
BsmtExposure     -     2.6027397260273974
BsmtFinType1     -     2.5342465753424657
BsmtFinType2     -     2.6027397260273974
Electrical     -     0.0684931506849315
FireplaceQu     -     47.26027397260274
GarageType     -     5.5479452054794525
GarageYrBlt     -     5.5479452054794525
GarageFinish     -     5.5479452054794525
GarageQual     -     5.5479452054794525
GarageCond     -     5.5479452054794525
PoolQC     -     99.52054794520548
Fence     -     80.75342465753425
MiscFeature     -     96.3013698630137


In [8]:
print("Percentage of null values for df_test")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    percentage_null_values = df_test.isnull().sum()*100/df_test.shape[0]
    
    for col,val in percentage_null_values.iteritems():
        if val!=0:
            print(col,"    -    ",val)
            

Percentage of null values for df_test
MSZoning     -     0.27416038382453733
LotFrontage     -     15.558601782042494
Alley     -     92.66620973269363
Utilities     -     0.13708019191226867
Exterior1st     -     0.06854009595613433
Exterior2nd     -     0.06854009595613433
MasVnrType     -     1.0966415352981493
MasVnrArea     -     1.0281014393420151
BsmtQual     -     3.015764222069911
BsmtCond     -     3.0843043180260454
BsmtExposure     -     3.015764222069911
BsmtFinType1     -     2.8786840301576424
BsmtFinSF1     -     0.06854009595613433
BsmtFinType2     -     2.8786840301576424
BsmtFinSF2     -     0.06854009595613433
BsmtUnfSF     -     0.06854009595613433
TotalBsmtSF     -     0.06854009595613433
BsmtFullBath     -     0.13708019191226867
BsmtHalfBath     -     0.13708019191226867
KitchenQual     -     0.06854009595613433
Functional     -     0.13708019191226867
FireplaceQu     -     50.03427004797807
GarageType     -     5.2090472926662095
GarageYrBlt     -     5.3461274

In [9]:
df = pd.concat([df_train,df_test])

C:\Users\kungw\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
df.columns

Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st',
       'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation',
       'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea',
       'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'Id', 'KitchenAbvGr',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig',
       'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning',
       'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'Po

In [13]:
threshold = 20
col_with_null_beyond_threshold = []
col_with_null_below_threshold = []

print("Percentage of null values for total df")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    percentage_null_values = df.isnull().sum()*100/df.shape[0]
    
    for col,val in percentage_null_values.iteritems():
        if col=='SalePrice':
            continue
        if val!=0:
            print(col,"    -    ",val)
            if val>threshold:
                col_with_null_beyond_threshold.append(col)
            else:
                col_with_null_below_threshold.append(col)

Percentage of null values for total df
Alley     -     93.21685508735868
BsmtCond     -     2.8091812264474134
BsmtExposure     -     2.8091812264474134
BsmtFinSF1     -     0.0342583076396026
BsmtFinSF2     -     0.0342583076396026
BsmtFinType1     -     2.7064063035286057
BsmtFinType2     -     2.7406646111682083
BsmtFullBath     -     0.0685166152792052
BsmtHalfBath     -     0.0685166152792052
BsmtQual     -     2.774922918807811
BsmtUnfSF     -     0.0342583076396026
Electrical     -     0.0342583076396026
Exterior1st     -     0.0342583076396026
Exterior2nd     -     0.0342583076396026
Fence     -     80.43850633778692
FireplaceQu     -     48.646796848235695
Functional     -     0.0685166152792052
GarageArea     -     0.0342583076396026
GarageCars     -     0.0342583076396026
GarageCond     -     5.447070914696814
GarageFinish     -     5.447070914696814
GarageQual     -     5.447070914696814
GarageType     -     5.378554299417609
GarageYrBlt     -     5.447070914696814
KitchenQ

In [14]:
df.shape

(2919, 81)

In [15]:
col_with_null_beyond_threshold

['Alley', 'Fence', 'FireplaceQu', 'MiscFeature', 'PoolQC']

In [16]:
df = df.drop(col_with_null_beyond_threshold,axis=1)

In [17]:
df.shape

(2919, 76)

In [18]:
df.iloc[df_train.shape[0]-1,:]['SalePrice']

147500.0

In [19]:
col_with_null_below_threshold

['BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtQual',
 'BsmtUnfSF',
 'Electrical',
 'Exterior1st',
 'Exterior2nd',
 'Functional',
 'GarageArea',
 'GarageCars',
 'GarageCond',
 'GarageFinish',
 'GarageQual',
 'GarageType',
 'GarageYrBlt',
 'KitchenQual',
 'LotFrontage',
 'MSZoning',
 'MasVnrArea',
 'MasVnrType',
 'SaleType',
 'TotalBsmtSF',
 'Utilities']

In [20]:
for col in col_with_null_below_threshold:
    df[col] = df[col].fillna(df[col].mode()[0])

In [21]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df.isna().sum())

1stFlrSF            0
2ndFlrSF            0
3SsnPorch           0
BedroomAbvGr        0
BldgType            0
BsmtCond            0
BsmtExposure        0
BsmtFinSF1          0
BsmtFinSF2          0
BsmtFinType1        0
BsmtFinType2        0
BsmtFullBath        0
BsmtHalfBath        0
BsmtQual            0
BsmtUnfSF           0
CentralAir          0
Condition1          0
Condition2          0
Electrical          0
EnclosedPorch       0
ExterCond           0
ExterQual           0
Exterior1st         0
Exterior2nd         0
Fireplaces          0
Foundation          0
FullBath            0
Functional          0
GarageArea          0
GarageCars          0
GarageCond          0
GarageFinish        0
GarageQual          0
GarageType          0
GarageYrBlt         0
GrLivArea           0
HalfBath            0
Heating             0
HeatingQC           0
HouseStyle          0
Id                  0
KitchenAbvGr        0
KitchenQual         0
LandContour         0
LandSlope           0
LotArea   

In [22]:
df.shape

(2919, 76)

In [23]:
categorical_columns = []
for col in df.columns:
    if df[col].dtype == object:
        categorical_columns.append(col)

In [24]:
for index,col in enumerate(categorical_columns):
    
    df1 = pd.get_dummies(df[col],drop_first = True)
    df.drop([col],axis=1,inplace=True)
    df = pd.concat([df,df1],axis=1)

        

In [25]:
df.iloc[df_train.shape[0]-1,:]['SalePrice']

147500.0

In [26]:
df = df.loc[:,~df.columns.duplicated()]


In [27]:
df.shape

(2919, 178)

In [28]:
df.iloc[df_train.shape[0]-1,:]['SalePrice']

147500.0

In [29]:
df.drop(['Id'],axis=1,inplace=True)

In [30]:
df.iloc[df_train.shape[0]-1,:]['SalePrice']

147500.0

In [31]:
'SalePrice' in df.columns

True

In [32]:
dfTrain = df.iloc[:df_train.shape[0],:]
dfTest = df.iloc[df_train.shape[0]:,:]

In [33]:
dfTrain['SalePrice']

0       208500.0
1       181500.0
2       223500.0
3       140000.0
4       250000.0
          ...   
1455    175000.0
1456    210000.0
1457    266500.0
1458    142125.0
1459    147500.0
Name: SalePrice, Length: 1460, dtype: float64

In [34]:
dfTest.drop(['SalePrice'],axis=1,inplace=True)

C:\Users\kungw\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [35]:
dfTrain['SalePrice']

0       208500.0
1       181500.0
2       223500.0
3       140000.0
4       250000.0
          ...   
1455    175000.0
1456    210000.0
1457    266500.0
1458    142125.0
1459    147500.0
Name: SalePrice, Length: 1460, dtype: float64

In [36]:
dfTest.shape

(1459, 176)

In [37]:
y = dfTrain['SalePrice']
x = dfTrain.drop(['SalePrice'],axis=1)

In [38]:
y

0       208500.0
1       181500.0
2       223500.0
3       140000.0
4       250000.0
          ...   
1455    175000.0
1456    210000.0
1457    266500.0
1458    142125.0
1459    147500.0
Name: SalePrice, Length: 1460, dtype: float64

In [39]:
import xgboost
regressor=xgboost.XGBRegressor()

In [42]:
'SalePrice' in x.columns

False

In [43]:
bst = regressor.fit(x,y)


In [45]:
'SalePrice' in dfTest.columns

False

In [46]:
y_pred = regressor.predict(dfTest)

In [ ]:
from sklearn.metrics import mean_squared_error
print()

In [48]:
pred=pd.DataFrame(y_pred)
sub_df=pd.read_csv('data/sample_submission.csv')
datasets=pd.concat([sub_df['Id'],pred],axis=1)
datasets.columns=['Id','SalePrice']
datasets.to_csv('sample_submission.csv',index=False)

In [30]:
# Alley : NA - No alley access
# BsmtQual : NA - No basement
# BsmtCond : NA - No basement
# BsmtExposure : NA - No basement
# BsmtFinType1 : NA - No basemen
# BsmtFinType2 : NA - No basemen
# FireplaceQu : NA - No fireplace
# GarageType : NA  - No Garage
# GarageFinish : NA  - No Garage
# GarageFinish : NA  - No Garage
# GarageCond : NA  - No Garage
# PoolQC : NA - No pool
# Fence : NA - No Fence
# MiscFeature : NA - No Other luxury

In [31]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [34]:
categorical_col = []
continous_col = []
for col in df.columns:
    if df[col].dtype == object:
        categorical_col.append(col)
    else:
        continous_col.append(col)

In [ ]:
#drop id since it is unique for each data point

In [ ]:
col_with_null_beyond_threshold